# 🎮 Chapter 3D-6: The First Person Camera

You have a cube. You can spin it. But you are **still an observer**.

To make a game, you need to **walk inside the world**.

This chapter connects **Pygame Input** (WASD + Mouse) to the **View Matrix**.

## 1. The Camera Class

A camera has two properties:
1.  **Position**: Where the player is standing (X, Y, Z).
2.  **Rotation**: Where the player is looking (Yaw, Pitch).

From these, we can calculate the **View Matrix**.

In [ ]:
import numpy as np
from pyrr import Matrix44, Vector3
import math

class Camera:
    def __init__(self):
        self.position = Vector3([0.0, 0.0, 3.0])
        self.yaw = -90.0   # Left/Right (degrees)
        self.pitch = 0.0   # Up/Down (degrees)
        
        self.speed = 2.5
        self.sensitivity = 0.1
        
        self.update_vectors()
    
    def update_vectors(self):
        """Calculate Forward, Right, and Up vectors from yaw and pitch."""
        front = Vector3([
            math.cos(math.radians(self.yaw)) * math.cos(math.radians(self.pitch)),
            math.sin(math.radians(self.pitch)),
            math.sin(math.radians(self.yaw)) * math.cos(math.radians(self.pitch))
        ])
        
        self.front = front.normalized
        self.right = Vector3.cross(self.front, Vector3([0, 1, 0])).normalized
        self.up = Vector3.cross(self.right, self.front).normalized
    
    def get_view_matrix(self):
        """Return the View Matrix for rendering."""
        return Matrix44.look_at(
            self.position,
            self.position + self.front,
            self.up
        )
    
    def process_keyboard(self, keys, dt):
        """Move the camera based on WASD input."""
        import pygame
        velocity = self.speed * dt
        
        if keys[pygame.K_w]:
            self.position += self.front * velocity
        if keys[pygame.K_s]:
            self.position -= self.front * velocity
        if keys[pygame.K_a]:
            self.position -= self.right * velocity
        if keys[pygame.K_d]:
            self.position += self.right * velocity
        if keys[pygame.K_SPACE]:
            self.position += Vector3([0, 1, 0]) * velocity
        if keys[pygame.K_LSHIFT]:
            self.position -= Vector3([0, 1, 0]) * velocity
    
    def process_mouse(self, xoffset, yoffset):
        """Rotate the camera based on mouse movement."""
        xoffset *= self.sensitivity
        yoffset *= self.sensitivity
        
        self.yaw += xoffset
        self.pitch += yoffset
        
        # Clamp pitch to prevent screen flip
        if self.pitch > 89.0:
            self.pitch = 89.0
        if self.pitch < -89.0:
            self.pitch = -89.0
        
        self.update_vectors()

## 2. Capturing Mouse Movement

Pygame can lock the mouse to the window and give us **relative movement**.

In [ ]:
import pygame

pygame.init()
screen = pygame.display.set_mode((800, 600), pygame.OPENGL | pygame.DOUBLEBUF)

# Lock the mouse to the window
pygame.event.set_grab(True)
pygame.mouse.set_visible(False)

camera = Camera()
clock = pygame.time.Clock()

running = True
while running:
    dt = clock.tick(60) / 1000.0  # Delta time in seconds
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
        # Unlock mouse on ESC key
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
        
        # Mouse movement
        if event.type == pygame.MOUSEMOTION:
            xoffset, yoffset = event.rel
            camera.process_mouse(xoffset, -yoffset)  # Inverted Y
    
    # Keyboard movement
    keys = pygame.key.get_pressed()
    camera.process_keyboard(keys, dt)
    
    # Get the updated view matrix
    view = camera.get_view_matrix()
    
    # ... render your scene with this view matrix ...

pygame.quit()

## 3. Putting It All Together

Combine the Camera with the Cube from Chapter 5.

In [ ]:
import pygame
import moderngl
import numpy as np
from pyrr import Matrix44

# Initialize
pygame.init()
screen = pygame.display.set_mode((800, 600), pygame.OPENGL | pygame.DOUBLEBUF)
ctx = moderngl.create_context()
ctx.enable(moderngl.DEPTH_TEST)

pygame.event.set_grab(True)
pygame.mouse.set_visible(False)

# ... (Load cube vertices and shaders from Chapter 5) ...

camera = Camera()
projection = Matrix44.perspective_projection(45.0, 800/600, 0.1, 100.0)

clock = pygame.time.Clock()
running = True

while running:
    dt = clock.tick(60) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
            running = False
        if event.type == pygame.MOUSEMOTION:
            camera.process_mouse(event.rel[0], -event.rel[1])
    
    keys = pygame.key.get_pressed()
    camera.process_keyboard(keys, dt)
    
    # Calculate MVP
    view = camera.get_view_matrix()
    model = Matrix44.identity()  # Cube at origin
    mvp = projection * view * model
    
    prog['mvp'].write(mvp.astype('f4').tobytes())
    
    ctx.clear(0.1, 0.1, 0.1)
    vao.render()
    
    pygame.display.flip()

pygame.quit()

## 🛠️ Challenge: The 3D Maze

You now have everything you need to build a first-person game.

1.  Create a list of cube positions (e.g., walls in a grid).
2.  In the loop, render each cube at its position using a Model Matrix.
3.  Use the camera to walk through the maze.

**Bonus:** Add collision detection so you can't walk through walls.

If you can navigate a 3D space you built from scratch, **you are a game developer**. 🎖️